# Requirements

- Python version >= 3.10
- `pip install pandas openpyxl pydantic`

In [36]:
import pandas as pd
from pydantic import BaseModel

In [49]:
df_p = pd.read_excel("US_Population.xlsx")
df_p.head()

,CITY,STATE,POPULATION
0,New York[c],NY,8804190
1,Los Angeles,CA,3898747
2,Chicago,IL,2746388
3,Houston,TX,2304580
4,Phoenix,AZ,1608139


In [66]:
df_i = pd.read_excel("US_Income.xlsx")
df_i.head()

,Column1,Column2,Column3,CITY_CODE,Column5,Column6,Column7,MEDIAN_INCOME,Column9,Column10,...,Column28,Column29,Column30,Column31,Column32,Column33,Column34,Column35,Column36,Column37
0,2,1,37,20500,63,1708,90.17,82543,91167,0.9054,...,9,0,1,0,0,0,0,0,0,0
1,2,2,12,33100,86,9904,98.03,44931,73392,0.6122,...,9,100,1,0,0,0,0,0,0,0
2,2,3,47,32820,157,6500,98.76,46375,72130,0.6429,...,9,100,1,0,1,0,0,1,0,0
3,2,4,45,24860,45,2821,29.70,96463,75341,1.2804,...,9,0,1,0,0,0,0,0,0,0
4,2,5,48,12420,209,10919,68.38,75045,105379,0.7121,...,9,0,1,0,0,0,0,0,0,0


In [ ]:
len(df_i[df_i['CITY_CODE'] == 24860])
df_i[df_i['CITY_CODE'] == 24860].iloc[0]['MEDIAN_INCOME']

np.float64(96463.0)

In [ ]:
with open('./MSA_Cities_2023.txt', encoding='utf-8', mode='r') as f:
    msa_2023_text = f.read()
msa_2023_list: list[str] = msa_2023_text.split('\n')

In [25]:
msa_2023_dict: dict[int, str] = {}
for line in msa_2023_list:
    if len(line.strip().split('=')) == 2:
        code_str, location = line.strip().split('=')
        code = int(code_str.strip())
        msa_2023_dict[code] = location.strip().replace('\'', '')

In [70]:
class City(BaseModel):
    code: int
    location: str
    population: int
    income: int

In [71]:
cities: list[City] = []
for index, row in df_p.iterrows():
    df_p_city: str = row['CITY']
    if df_p_city.endswith(']'):
        df_p_city = df_p_city[:-3]
    found = False
    for code, location in msa_2023_dict.items():
        if df_p_city.lower() in location.lower() and row['STATE'].lower() in location.lower():
            if len(df_i[df_i['CITY_CODE'] == 24860]) > 0:
                income = int(df_i[df_i['CITY_CODE'] == 24860].iloc[0]['MEDIAN_INCOME'])
            city = City(code=code, location=location, population=row['POPULATION'], income=income)
            cities.append(city)
            found = True
    if not found:
        print(f'{df_p_city}, {row['STATE']} not in msa_2023!')

Irvine, CA not in msa_2023!
Santa Ana, CA not in msa_2023!
Saint Paul, MN not in msa_2023!
Plano, TX not in msa_2023!
North Las Vegas, NV not in msa_2023!
Gilbert, AZ not in msa_2023!
Irving, TX not in msa_2023!
Chesapeake, VA not in msa_2023!
Glendale, AZ not in msa_2023!
Scottsdale, AZ not in msa_2023!
Garland, TX not in msa_2023!
Fremont, CA not in msa_2023!
Frisco, TX not in msa_2023!
Santa Clarita, CA not in msa_2023!
Hialeah, FL not in msa_2023!
Fontana, CA not in msa_2023!
McKinney, TX not in msa_2023!
Moreno Valley, CA not in msa_2023!
Yonkers, NY not in msa_2023!
Grand Prairie, TX not in msa_2023!
Peoria, AZ not in msa_2023!
Overland Park, KS not in msa_2023!
Huntington Beach, CA not in msa_2023!
Tempe, AZ not in msa_2023!
Glendale, CA not in msa_2023!
Elk Grove, CA not in msa_2023!
Aurora, IL not in msa_2023!
Rancho Cucamonga, CA not in msa_2023!
Pembroke Pines, FL not in msa_2023!
Oceanside, CA not in msa_2023!
Garden Grove, CA not in msa_2023!
Palmdale, CA not in msa_2023!


In [72]:
import json
json_data = json.dumps([city.model_dump() for city in cities])

with open('./cities.json', encoding='utf-8', mode='w') as f:
    f.write(json_data)